In [2]:
import json
import numpy as np
import os
threshold = 0.75
path = '/home/banyh2000/odfn/wrapup_data/handcraft'
names = os.listdir(path)
for name in names:
    with open(os.path.join(path, name)) as f:
        data = json.load(f)
        data = np.array(data)
    length = data.shape[0]
    print(name)
    print(np.sum(data>threshold)/length)


output_functional_10.json
0.49
output_functional_15.json
0.81
output_functional_8.json
0.335
output_re.json
0.085
output_shift_0.8.json
0.085
output_shift_1.2.json
0.29
output_shift_1.5.json
0.9
output_natural.json
0.445
output_random.json
0.01
weak.json
0.25
weak_19990.json
0.17
weak_19999.json
0.12


In [1]:


import random
import numpy as np
from typing import TypeVar
T = TypeVar('T')
from PIL import Image
from pathlib import Path
import os
from tqdm import tqdm
from matplotlib import pyplot as plt


# compute if the p percent of the generated bounding box is in the original bounding box
def Con50(bounding_box_1,bounding_box_2):
    bounding_box_2 = [bounding_box_2[0],bounding_box_2[1],bounding_box_2[2]-bounding_box_2[0],bounding_box_2[3]-bounding_box_2[1]]
    x1 = max(bounding_box_1[0], bounding_box_2[0])
    y1 = max(bounding_box_1[1], bounding_box_2[1])
    x2 = min(bounding_box_1[0] + bounding_box_1[2], bounding_box_2[0] + bounding_box_2[2])
    y2 = min(bounding_box_1[1] + bounding_box_1[3], bounding_box_2[1] + bounding_box_2[3])
    w = max(0, x2 - x1)
    h = max(0, y2 - y1)
    intersection = w * h
    iou = intersection / (bounding_box_1[2] * bounding_box_1[3])
    return iou
    
    
    

bounding_box = [10,30,24,24]
bounding_box_image = [value * 8 for value in bounding_box]

values = []
for i in range(200):

    x,y = random.randint(0,512),random.randint(0,512)
    i,j = random.randint(0,512),random.randint(0,512)
    if x > i:
        x,i = i,x
    if y > j:
        y,j = j,y
    bounding_box_generated = [x,y,i-x,j-y]
    
    iou = Con50(bounding_box_image,bounding_box_generated)
    values.append(iou)
    import json
    with open('/home/banyh2000/odfn/wrapup_data/handcraft/output_random.json','w') as f:
        json.dump(values,f)


In [4]:
import os
os.chdir('/home/banyh2000/odfn')
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from scripts.models.diffuserpipeline import StableDiffusionPipeline
import torch
import random
import numpy as np
from typing import TypeVar
T = TypeVar('T')
from PIL import Image
from pathlib import Path
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from mmdet.apis import DetInferencer
from scripts.utils.utils_odfn import variance_5_class_index_sorted, seeds_plus, seeds_plus_dict,variance_index_sorted
inferencer = DetInferencer(model='rtmdet-ins_l_8xb32-300e_coco')

def auto_device(obj: T = torch.device('cpu')) -> T:
    if isinstance(obj, torch.device):
        return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.is_available():
        return obj.to('cuda')

def set_seed(seed: int) -> torch.Generator:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    gen = torch.Generator(device=auto_device())
    gen.manual_seed(seed)

    return gen

prompt = "A sports ball is caught in a fence."

model_id = 'stabilityai/stable-diffusion-2-base'
device = 'cuda'
pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True).to(device)
dict_ = {}
for i in range(0,20000,1000):
    seed = seeds_plus[variance_index_sorted[i]]
    with torch.no_grad():
        out = pipe(prompt=prompt, generator=set_seed(seed))
        out.images[0].save(f'/home/banyh2000/odfn/wrapup_data/handcraft/{i}_1.png')
        image = np.array(out.images[0])
        results = inferencer(image)
        bounding_box_generated = results['predictions'][0]['bboxes'][0]
        bounding_box = [value/8 for value in bounding_box_generated]
        dict_[i] = bounding_box
        
import json
with open('/home/banyh2000/odfn/wrapup_data/handcraft/bounding boxes_1.json','w') as f:
    json.dump(dict_,f)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet-ins_l_8xb32-300e_coco/rtmdet-ins_l_8xb32-300e_coco_20221124_103237-78d1d652.pth


Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.
100%|██████████| 50/50 [00:04<00:00, 11.46it/s]


100%|██████████| 50/50 [00:04<00:00, 11.43it/s]


100%|██████████| 50/50 [00:04<00:00, 11.42it/s]


100%|██████████| 50/50 [00:04<00:00, 11.41it/s]


...

100%|██████████| 50/50 [00:04<00:00, 11.36it/s]


100%|██████████| 50/50 [00:04<00:00, 11.37it/s]


100%|██████████| 50/50 [00:04<00:00, 11.37it/s]


100%|██████████| 50/50 [00:04<00:00, 11.36it/s]


100%|██████████| 50/50 [00:04<00:00, 11.35it/s]
